In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.set_option('display.max_colwidth', None)
!pip install nbimporter
!pip install nlp
!pip install text2emotion
import text2emotion as te
!pip uninstall emoji
!pip install emoji==1.7
!pip install text2emotion
            


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Found existing installation: emoji 1.7.0
Uninstalling emoji-1.7.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/emoji-1.7.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/emoji/*
Proceed (y/n)? y
  Successfully uninstalled emoji-1.7.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached emoji-1.7.0-py3-none-any.whl


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
df = pd.read_json('scripts_grouped.json')

In [4]:
#df[['dialogue', 'script_length']].head()

In [5]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
sw = stopwords.words('english')

def clean(text):
    lower_case = text.lower()
    tokenized = word_tokenize(lower_case)
    last_item = tokenized[:-1]
    script_length = len(last_item)
    
    first = ['i', 'we', 'me', 'us', 'our', 'my']
    second = ['you', 'your']
    third = ['he', 'him', 'his', 'she', 'her']
    first_person = len([word for word in last_item if word in first])
    second_person = len([word for word in last_item if word in second])
    third_person = len([word for word in last_item if word in third])
    first_person_percent = first_person/(first_person+second_person+third_person) * 100
    second_person_percent = second_person/(first_person+second_person+third_person) * 100
    third_person_percent = third_person/(first_person+second_person+third_person) * 100
       
    no_stopwords = [word for word in last_item if word not in sw]
    stopwords = [word for word in last_item if word in sw] 
    alpha_only = [word for word in no_stopwords if word.isalpha()]
    alnums = [word for word in no_stopwords if word.isalnum()]
    num_only = [word for word in alnums if word not in alpha_only]
    
    
    stop_words_percent = len(stopwords)/len(last_item) * 100
    no_stopwords_percent = len(no_stopwords)/len(last_item) * 100
    number_percent = len(num_only)/len(last_item) * 100
    
    str1 = " "
    cleaned = str1.join(last_item)
    
    lex_diversity = len(set(last_item))/len(last_item)
    
    return script_length, first_person_percent, second_person_percent, third_person_percent,\
stop_words_percent, no_stopwords_percent, cleaned, lex_diversity

df['script_length'], df['first_person_percent'], df['second_person_percent'], df['third_person_percent'],\
df['stop_words_percent'], df['no_stopwords_percent'], df['dialogue_clean'], df['lex_diversity'] = zip(*df['dialogue'].apply(clean))

In [6]:
from textblob import TextBlob
# 2) subjectivity: objective vs. subjective (+0.0 => +1.0)
df['subjectivity'] = df['dialogue_clean'].apply(lambda x: TextBlob(x).subjectivity)
df['polarity'] = df['dialogue_clean'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [7]:
df_truncated = df[['script_length', 'first_person_percent', 'second_person_percent', \
                   'third_person_percent', 'stop_words_percent', 'no_stopwords_percent', 'lex_diversity', \
                  'subjectivity', 'polarity']]
df_truncated.head()

,script_length,first_person_percent,second_person_percent,third_person_percent,stop_words_percent,no_stopwords_percent,lex_diversity,subjectivity,polarity
0,4094,40.572792,43.675418,15.751790,38.470933,61.529067,0.185882,0.519149,0.122309
10170,3847,49.397590,31.566265,19.036145,35.846114,64.153886,0.195737,0.513319,0.210641
10455,4946,47.833622,39.688042,12.478336,38.536191,61.463809,0.146583,0.499423,0.130353
10852,3922,54.741379,32.543103,12.715517,37.021928,62.978072,0.173126,0.431258,0.030128
1087,4129,54.126214,35.194175,10.679612,39.065149,60.934851,0.195447,0.511490,0.088897


In [ ]:
import text2emotion as te
import re 
!pip install emot
from emoji import EMOJI_DATA
import nltk
nltk.download('omw-1.4')

def emotion_score(text):
    emotion_score = te.get_emotion(text)
    emotion_happy = emotion_score.get('Happy')
    emotion_angry = emotion_score.get('Angry')
    emotion_surprise = emotion_score.get('Surprise')
    emotion_sad = emotion_score.get('Sad')
    emotion_fear = emotion_score.get('Fear')
    
    return emotion_happy, emotion_angry, emotion_surprise, emotion_sad, emotion_fear

df['emotion_happy'], df['emotion_angry'], df['emotion_surprise'], df['emotion_sad'], \
df['emotion_fear'] = zip(*df['dialogue'].apply(emotion_score))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
df[['emotion_happy', 'emotion_angry', 'emotion_surprise', 'emotion_sad', 'emotion_fear']]

# New Section